In [1]:
import h5py
import numpy as np
import os.path
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import os
from PIL import Image
import imageio.v2 as imageio
import pandas as pd
import matplotlib.pyplot as plt
pio.renderers.default = 'browser'

## Read HDF5 file

In [2]:
f = h5py.File("fileout.h5", "r")
for key in f.keys():
    print(key, ":", type(f[key])) #Names of the root level object names in HDF5 file - can be groups or datasets.

B0x : <class 'h5py._hl.dataset.Dataset'>
B0y : <class 'h5py._hl.dataset.Dataset'>
B0z : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint00400 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint00800 : <class 'h5py._hl.dataset.Dataset'>
config : <class 'h5py._hl.group.Group'>
detAnt_01 : <class 'h5py._hl.group.Group'>
detAnt_02 : <class 'h5py._hl.group.Group'>
n_e : <class 'h5py._hl.dataset.Dataset'>


In [3]:
for key in f["config"].keys():
    print(key, ":",type(f["config"][key])) #Names of the root level object names in HDF5 file - can be groups or datasets.

N_x : <class 'h5py._hl.dataset.Dataset'>
N_y : <class 'h5py._hl.dataset.Dataset'>
N_z : <class 'h5py._hl.dataset.Dataset'>
ant_w0x : <class 'h5py._hl.dataset.Dataset'>
ant_w0y : <class 'h5py._hl.dataset.Dataset'>
ant_x : <class 'h5py._hl.dataset.Dataset'>
ant_y : <class 'h5py._hl.dataset.Dataset'>
ant_z : <class 'h5py._hl.dataset.Dataset'>
d_absorb : <class 'h5py._hl.dataset.Dataset'>
period : <class 'h5py._hl.dataset.Dataset'>
z2waist : <class 'h5py._hl.dataset.Dataset'>


## Plot data HDF5(2D): Electric field

'aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
             'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
             'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
             'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
             'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
             'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
             'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
             'orrd', 'oryel', 'oxy', 'peach', 'phase', 'picnic', 'pinkyl',
             'piyg', 'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn',
             'puor', 'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu',
             'rdgy', 'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar',
             'spectral', 'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn',
             'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
             'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
             'ylorrd'

In [5]:
x_slice = 100
output_gif = "simulation_PML_test.gif"
images = []

for i in range(1,51):
    if(i <= 6):
        E_field_ds = f[f'E_abs__tint000{16*i}'][:]
    else:
        E_field_ds = f[f'E_abs__tint00{16*i}'][:]
    sliced_data = E_field_ds[x_slice, :, :]

    fig = go.Figure( data=go.Heatmap(
        z = sliced_data,     # Flattened volume data
        colorscale='delta',                 # Choose a colorscale
        zmin=0,
        zmax=2
    ))
    
    # Update layout
    fig.update_layout(
        title=f"Abs(E) = {i}",
        xaxis_title="Z",
        yaxis_title="Y",
    )
    
    # Show the plot
    #pio.write_image(fig, f"{folder_path}/E_field(t)={i}", format="png")
    img_bytes = fig.to_image(format="png",width=1600,height=1200)
    img = imageio.imread(img_bytes)
    images.append(img)
    #fig.show()

imageio.mimsave(f'{output_gif}', images, duration=200)

In [11]:
x_slice = 100
output_gif = "simulation_PML40_testLog.gif"
images = []
offset = 1e-16

for i in range(1,51):
    if(i <= 6):
        E_field_ds = f[f'E_abs__tint000{16*i}'][:]
    else:
        E_field_ds = f[f'E_abs__tint00{16*i}'][:]
    sliced_data = E_field_ds[x_slice, :, :]
    z_log = np.log(sliced_data + offset)

    fig = go.Figure( data=go.Heatmap(
        z = z_log,     # Flattened volume data
        #z = sliced_data,
        colorscale='jet',                 # Choose a colorscale
        zmin=np.min( np.log10(1e-10) ),
        zmax=np.max( np.log10(2) ),
        colorbar=dict(
            title="logAbs(E)",
            tickvals= np.logspace( 1e-10, 2, 5),
            ticktext= [f"{1e-10}","0.1","1","2",f"{2}"]
        )
    ))
    
    # Update layout
    fig.update_layout(
        title=f"Abs(E) = {i}",
        xaxis_title="Z",
        yaxis_title="Y",
    )
    
    # Show the plot
    #pio.write_image(fig, f"{folder_path}/E_field(t)={i}", format="png")
    img_bytes = fig.to_image(format="png",width=1600,height=1200)
    img = imageio.imread(img_bytes)
    images.append(img)
    #fig.show()

imageio.mimsave(f'{output_gif}', images, duration=200)

In [12]:
sliced_data = sliced_data.flatten()
unique_values =np.unique(sliced_data)
print( unique_values )

[0.00000000e+00 6.72891353e-10 1.09394139e-09 ... 1.52418850e+00
 1.54952798e+00 1.55883218e+00]


## Plot data HDF5: Electric Field 3D

In [8]:
grid = f["E_abs__tint00800"][:]

In [9]:
print(f"Shape: {grid.shape}")  # Shape of the dataset (dimensions)
print(f"Data Type (dtype): {grid.dtype}")  # Data type of the elements
print(f"Size: {grid.size}")  # Total number of elements in the dataset

Shape: (200, 150, 100)
Data Type (dtype): float64
Size: 3000000


In [10]:
# Generate the coordinate grid for the 3D volume (X, Y, Z)
x = grid.shape[0]
y = grid.shape[1]
z = grid.shape[2]
X, Y, Z = np.mgrid[ 0:x:x*1j, 0:y:y*1j, 0:z:z*1j ]

In [13]:
# Create a 3D Volume Plot using Plotly
fig = go.Figure(data=go.Volume(
    x=X.flatten(),                 # X-coordinates
    y=Y.flatten(),                 # Y-coordinates
    z=Z.flatten(),                 # Z-coordinates
    value = grid.flatten(),     # Flattened volume data
    isomin=0.5,                    # Minimum value to display
    isomax=np.max(grid),        # Maximum value to display
    opacity=0.3,                   # Adjust opacity (between 0 and 1) for better visualization
    surface_count=20,              # Number of volume surfaces to display
    colorscale='jet',           # Choose a colorscale
))

# Update layout
fig.update_layout(scene=dict(
    xaxis_title='X Axis',
    yaxis_title='Y Axis',
    zaxis_title='Z Axis'
))

# Show the plot
fig.show()

In [4]:
E_field_ds = f['E_abs__tint64'][:]
print(f"Shape: {E_field_ds.shape}")  # Shape of the dataset (dimensions)
print(f"Data Type (dtype): {E_field_ds.dtype}")  # Data type of the elements
print(f"Size: {E_field_ds.size}")  # Total number of elements in the dataset

Shape: (200, 150, 100)
Data Type (dtype): float64
Size: 3000000


## Magnetic field

In [16]:
B_field_ds = E_f['B_abs'][:]
print(f"Shape: {B_field_ds.shape}")  # Shape of the dataset (dimensions)
print(f"Data Type (dtype): {B_field_ds.dtype}")  # Data type of the elements
print(f"Size: {B_field_ds.size}")  # Total number of elements in the dataset

Shape: (150, 150, 200)
Data Type (dtype): float64
Size: 4500000


In [17]:
# Create a 3D Volume Plot using Plotly
fig = go.Figure(data=go.Volume(
    x=X.flatten(),                 # X-coordinates
    y=Y.flatten(),                 # Y-coordinates
    z=Z.flatten(),                 # Z-coordinates
    value = B_field_ds.flatten(),     # Flattened volume data
    isomin=0.2,                    # Minimum value to display
    isomax=np.max(B_field_ds),        # Maximum value to display
    opacity=0.3,                   # Adjust opacity (between 0 and 1) for better visualization
    surface_count=10,              # Number of volume surfaces to display
    colorscale='plasma',           # Choose a colorscale
))

# Update layout
fig.update_layout(scene=dict(
    xaxis_title='X Axis',
    yaxis_title='Y Axis',
    zaxis_title='Z Axis'
))

# Show the plot
fig.show()

### Plot PML boundary

In [7]:
E_field_ds = f[f'E_abs__tint00800'][:]
x_data = E_field_ds[:, 75, 50]
y_data = E_field_ds[100, :, 50]
z_data = E_field_ds[100, 75,:]
x_pos = np.linspace(0,200,len(x_data) )
x_pos

array([  0.        ,   1.00502513,   2.01005025,   3.01507538,
         4.0201005 ,   5.02512563,   6.03015075,   7.03517588,
         8.04020101,   9.04522613,  10.05025126,  11.05527638,
        12.06030151,  13.06532663,  14.07035176,  15.07537688,
        16.08040201,  17.08542714,  18.09045226,  19.09547739,
        20.10050251,  21.10552764,  22.11055276,  23.11557789,
        24.12060302,  25.12562814,  26.13065327,  27.13567839,
        28.14070352,  29.14572864,  30.15075377,  31.15577889,
        32.16080402,  33.16582915,  34.17085427,  35.1758794 ,
        36.18090452,  37.18592965,  38.19095477,  39.1959799 ,
        40.20100503,  41.20603015,  42.21105528,  43.2160804 ,
        44.22110553,  45.22613065,  46.23115578,  47.2361809 ,
        48.24120603,  49.24623116,  50.25125628,  51.25628141,
        52.26130653,  53.26633166,  54.27135678,  55.27638191,
        56.28140704,  57.28643216,  58.29145729,  59.29648241,
        60.30150754,  61.30653266,  62.31155779,  63.31

In [12]:
fig = px.scatter(z_data)
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of [0] but received: Z Position

In [2]:
i = 1
data = pd.read_csv(f"../Antenna_detector/PML_energy/Energy_density.txt")
data

,Pos,Value
0,0,0.000000e+00
1,1,1.343000e-07
2,2,5.176500e-06
3,3,4.711900e-05
4,4,1.967513e-04
...,...,...
796,796,2.035447e+04
797,797,2.057337e+04
798,798,2.090130e+04
799,799,2.114615e+04


In [6]:
plt.plot(data["Pos"], data["Value"],"navy",)
plt.title("Energy Density (full grid)")
plt.xlabel("Simulation Time")
plt.ylabel(r"Energy ($E^2/2 + B^2/2$)")
plt.savefig("EnergyDensity_PML.png", dpi=600)
plt.show()